In [1]:
!pip install -q pandas matplotlib datasets tqdm seqeval scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 10.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
bigframes 1.42.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.9.0.13 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cudnn-cu12==9.1.0.70; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cudnn-cu12 9.3.0.75 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cufft-cu12==11.2.1.3; platform_system == "Lin

# Preparations

## Import Libraries


In [2]:
import os
import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader 
from transformers import (
    AutoTokenizer, 
    AutoModelForTokenClassification,
    get_linear_schedule_with_warmup,
    AutoConfig,
)
from sklearn.metrics import accuracy_score, f1_score
from seqeval.metrics import classification_report, precision_score, recall_score, f1_score
from datasets import load_dataset, DatasetDict, Dataset
from torch.optim import AdamW
from tqdm import tqdm
from itertools import product
import json


## Set variables and Seed


In [3]:
seed = 42

np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

In [4]:
NER_TAG = ['B-LOC', 'I-LOC', 'B-ORG', 'I-ORG', 'B-PER', 'I-PER', 'O']
POS_TAG = ['PROPN', 'AUX', 'NUM', 'NOUN', 'ADP', 'PRON', 'VERB', 'ADV', 
           'ADJ', 'PUNCT', 'DET', 'PART', 'SCONJ', 'CCONJ', 'SYM', 'X']

In [5]:
label2id = {label: id for id, label in enumerate(NER_TAG)}
id2label = {id: label for label, id in label2id.items()}


In [6]:
def read_data(file_path):
    with open(file_path, encoding="utf-8") as f:
        lines = f.read().split("\n")
    
    data = []
    sentence = []
    id = 1
    for line in lines:
        if not line.strip():
            if sentence:
                data.append(sentence)
                sentence = []
        else:
            word, pos, tag = line.split()
            sentence.append((word, pos, tag))
            
    if sentence:
        data.append(sentence)

    # Create a dict for dataset
    raw_data_dict = {}
    
    for idx in range(len(data)):
    # for idx in range(20):
        raw_data_dict[idx] = {}
    
        words = []
        ner_tags = []
        pos_tags = []
        
        for i in range(len(data[idx])):
            words.append(data[idx][i][0])
            pos_tags.append(data[idx][i][1])
            ner_tags.append(data[idx][i][2])

        
        raw_data_dict[idx]['words'] = words
        raw_data_dict[idx]['ner_tags'] = ner_tags
        raw_data_dict[idx]['pos_tags'] = pos_tags
        
    # Convert raw_data to a list of dictionaries
    data_list = []
    for idx, data in raw_data_dict.items():
        data_list.append({
            'id': idx,
            'words': data['words'],
            'ner_tags': data['ner_tags'],
            'pos_tags': data['pos_tags'],
            'chunk_tags': []  # Placeholder, as your data doesn't have chunk_tags
        })
    return data_list

## Read Dataset


In [7]:
train_data = read_data('/kaggle/input/idner-news-2k/train.txt')
test_data = read_data('/kaggle/input/idner-news-2k/test.txt')
dev_data = read_data('/kaggle/input/idner-news-2k/valid.txt')
# train_data[:2]


In [8]:
train_dataset = Dataset.from_dict({k: [d[k] for d in train_data] for k in train_data[0]})
test_dataset = Dataset.from_dict({k: [d[k] for d in test_data] for k in test_data[0]})
dev_dataset = Dataset.from_dict({k: [d[k] for d in dev_data] for k in dev_data[0]})

# Create a DatasetDict
raw_data = DatasetDict({"train": train_dataset, "test": test_dataset, "dev": dev_dataset})
raw_data


DatasetDict({
    train: Dataset({
        features: ['id', 'words', 'ner_tags', 'pos_tags', 'chunk_tags'],
        num_rows: 1464
    })
    test: Dataset({
        features: ['id', 'words', 'ner_tags', 'pos_tags', 'chunk_tags'],
        num_rows: 509
    })
    dev: Dataset({
        features: ['id', 'words', 'ner_tags', 'pos_tags', 'chunk_tags'],
        num_rows: 367
    })
})

# Function Definitions

## Config


In [9]:
# Constants and configuration
def get_config():
    return {
        "max_length": 128,
        "batch_size": 16,
        "learning_rate": 2e-5,
        "epochs": 5,
        "warmup_steps": 500,
        "models": ["indobenchmark/indobert-base-p1", "xlm-roberta-base", "cahya/xlm-roberta-base-indonesian-NER"],
        "device": torch.device("cuda" if torch.cuda.is_available() else "cpu")
    }

## Tokenize


In [10]:
# Convert dataset to features
def tokenize_and_align_labels(tokenizer, examples, max_length):
    tokenized_inputs = []
    
    for example in examples:
        words = example["words"]
        labels = [label2id[label] for label in example["ner_tags"]]
        
        # Tokenize word by word to ensure alignment
        word_ids = []
        token_ids = []
        attention_mask = []
        label_ids = []
        
        for word_idx, (word, label) in enumerate(zip(words, labels)):
            # Tokenize the word and count # of tokens
            word_tokens = tokenizer.tokenize(word)
            if not word_tokens:
                word_tokens = [tokenizer.unk_token]
                
            # Add tokenized word to output
            token_ids.extend(tokenizer.convert_tokens_to_ids(word_tokens))
            attention_mask.extend([1] * len(word_tokens))
            
            # Add the word_id for each token
            word_ids.extend([word_idx] * len(word_tokens))
            
        # Handle truncation
        if len(token_ids) > max_length - 2:  # Account for [CLS] and [SEP]
            token_ids = token_ids[:max_length - 2]
            attention_mask = attention_mask[:max_length - 2]
            word_ids = word_ids[:max_length - 2]
        
        # Add special tokens
        token_ids = [tokenizer.cls_token_id] + token_ids + [tokenizer.sep_token_id]
        attention_mask = [1] + attention_mask + [1]
        word_ids = [None] + word_ids + [None]  # None for special tokens
        
        # Create aligned labels
        label_ids = [-100] * len(token_ids)  # Initialize with -100 (ignored in loss)
        
        for idx, word_id in enumerate(word_ids):
            if word_id is not None:
                label_ids[idx] = labels[word_id]
        
        # Pad to max_length
        padding_length = max_length - len(token_ids)
        if padding_length > 0:
            token_ids = token_ids + ([tokenizer.pad_token_id] * padding_length)
            attention_mask = attention_mask + ([0] * padding_length)
            label_ids = label_ids + ([-100] * padding_length)
        
        tokenized_inputs.append({
            "input_ids": token_ids,
            "attention_mask": attention_mask,
            "labels": label_ids,
            "word_ids": word_ids
        })
    
    return tokenized_inputs


## Prepare dataset


In [11]:
def prepare_datasets(dataset, tokenizer, config):
    tokenized_datasets = {}
    
    for split in dataset.keys():
        print(f"Preparing {split} dataset...")
        tokenized_datasets[split] = tokenize_and_align_labels(
            tokenizer, 
            dataset[split], 
            config["max_length"]
        )
    
    return tokenized_datasets


## Dataloaders


In [12]:
def create_dataloaders(tokenized_datasets, config):
    dataloaders = {}
    
    for split, dataset in tokenized_datasets.items():
        # Convert to PyTorch tensors
        tensor_dataset = []
        for item in dataset:
            tensor_dataset.append({
                "input_ids": torch.tensor(item["input_ids"]),
                "attention_mask": torch.tensor(item["attention_mask"]),
                "labels": torch.tensor(item["labels"])
            })
        
        # Create DataLoader
        shuffle = (split == "train")
        dataloaders[split] = DataLoader(
            tensor_dataset,
            batch_size=config["batch_size"],
            shuffle=shuffle
        )
    
    return dataloaders


## Train functions


In [13]:
# Training function
def train_model(model, dataloaders, optimizer, scheduler, config, model_name):
    device = config["device"]
    model.to(device)
    
    print(f"Training on {device}")
    best_f1 = 0
    
    for epoch in range(config["epochs"]):
        print(f"\nEpoch {epoch+1}/{config['epochs']}")
        
        # Training
        model.train()
        train_loss = 0
        progress_bar = tqdm(dataloaders["train"], desc=f"Training")
        
        for batch in progress_bar:
            # Move batch to device
            batch = {k: v.to(device) for k, v in batch.items()}
            
            # Zero gradients
            optimizer.zero_grad()
            
            # Forward pass
            outputs = model(**batch)
            loss = outputs.loss
            
            # Backward pass
            loss.backward()
            
            # Update
            optimizer.step()
            scheduler.step()
            
            # Track loss
            train_loss += loss.item()
            progress_bar.set_postfix({"loss": loss.item()})
        
        avg_train_loss = train_loss / len(dataloaders["train"])
        print(f"Average training loss: {avg_train_loss:.4f}")
        
        # Validation
        val_metrics = evaluate_model(model, dataloaders["dev"], config)
        
        # Save best model
        if val_metrics["f1"] > best_f1:
            best_f1 = val_metrics["f1"]
            print(f"New best F1: {best_f1:.4f} - Saving model")
            if not os.path.exists("models"):
                os.makedirs("models")
            torch.save(model.state_dict(), f"models/best_model_{model_name.replace('/', '_')}.pt")
        
        print(f"Validation Metrics: Precision={val_metrics['precision']:.4f}, Recall={val_metrics['recall']:.4f}, F1={val_metrics['f1']:.4f}, Exact Match= {val_metrics['exact_match']:.4f}")



## Evaluate functions


In [14]:
# Evaluation function
def evaluate_model(model, dataloader, config):
    device = config["device"]
    model.eval()
    
    all_predictions = []
    all_labels = []
    
    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Evaluating"):
            # Move batch to device
            batch = {k: v.to(device) for k, v in batch.items()}
            
            # Forward pass
            outputs = model(**batch)
            predictions = torch.argmax(outputs.logits, dim=2)
            
            # Extract predictions and labels, removing padding and special tokens
            for pred, label, mask in zip(predictions, batch["labels"], batch["attention_mask"]):
                pred = pred.cpu().numpy()
                label = label.cpu().numpy()
                mask = mask.cpu().numpy()
                
                true_labels = []
                true_preds = []
                
                for p, l, m in zip(pred, label, mask):
                    if m == 1 and l != -100:  # Skip padding and special tokens
                        true_labels.append(id2label[l])
                        true_preds.append(id2label[p])
                
                if true_labels:  # Only add non-empty sequences
                    all_labels.append(true_labels)
                    all_predictions.append(true_preds)
    
    # Calculate exact match
    exact_matches = sum(
        1 for preds, labels in zip(all_predictions, all_labels) if preds == labels
    )
    exact_match_score = exact_matches / len(all_labels) if all_labels else 0.0
    
    # Calculate metrics
    metrics = {
        "precision": precision_score(all_labels, all_predictions),
        "recall": recall_score(all_labels, all_predictions),
        "f1": f1_score(all_labels, all_predictions),
        "exact_match": exact_match_score
    }


    
    return metrics


## Predict funtions (single text)


In [15]:
# Inference function for a single text
def predict_entities(text, model, tokenizer, config):
    device = config["device"]
    model.to(device)
    model.eval()
    
    # Tokenize text
    words = text.split()
    word_tokens = []
    word_ids = []
    
    # Tokenize word-by-word
    for i, word in enumerate(words):
        tokens = tokenizer.tokenize(word)
        if not tokens:
            tokens = [tokenizer.unk_token]
        word_tokens.extend(tokens)
        word_ids.extend([i] * len(tokens))
    
    # Handle truncation
    if len(word_tokens) > config["max_length"] - 2:
        word_tokens = word_tokens[:config["max_length"] - 2]
        word_ids = word_ids[:config["max_length"] - 2]
    
    # Add special tokens
    encoded = tokenizer.encode_plus(
        word_tokens,
        is_split_into_words=True,
        add_special_tokens=True,
        max_length=config["max_length"],
        padding="max_length",
        truncation=True,
        return_tensors="pt"
    )
    
    # Predict
    with torch.no_grad():
        encoded = {k: v.to(device) for k, v in encoded.items()}
        outputs = model(**encoded)
        predictions = torch.argmax(outputs.logits, dim=2)[0].cpu().numpy()
    
    # Process predictions
    previous_word_id = None
    entities = []
    current_entity = {"text": "", "label": "", "start": -1, "end": -1}
    
    special_tokens_mask = np.array(
        [1] + [0] * len(word_ids) + [1] + [1] * (config["max_length"] - len(word_ids) - 2)
    )
    
    for i, (prediction, word_id, is_special) in enumerate(
        zip(predictions, [None] + word_ids + [None] + [None] * (config["max_length"] - len(word_ids) - 2), special_tokens_mask)
    ):
        # Skip special tokens and padding
        if is_special or word_id is None:
            continue
        
        label = id2label.get(prediction, "O")
        
        # If we've moved to a new word
        if previous_word_id != word_id:
            # If we were building an entity, finalize it
            if current_entity["label"] and current_entity["label"] != "O":
                entities.append(current_entity.copy())
            
            # Start a new word
            previous_word_id = word_id
            
            # Check if this word starts a new entity
            if label.startswith("B-") or (label != "O" and not current_entity["label"]):
                current_entity = {
                    "text": words[word_id],
                    "label": label[2:] if label.startswith("B-") else label,
                    "start": word_id,
                    "end": word_id
                }
            elif label.startswith("I-") and current_entity["label"] == label[2:]:
                # Continue current entity
                current_entity["text"] += " " + words[word_id]
                current_entity["end"] = word_id
            elif label == "O":
                current_entity = {"text": "", "label": "", "start": -1, "end": -1}
    
    # Add the last entity if it exists
    if current_entity["label"] and current_entity["label"] != "O":
        entities.append(current_entity)
    
    return entities


# Proceed training


In [16]:
# Load configuration
config = get_config()
config

{'max_length': 128,
 'batch_size': 16,
 'learning_rate': 2e-05,
 'epochs': 5,
 'warmup_steps': 500,
 'models': ['indobenchmark/indobert-base-p1',
  'xlm-roberta-base',
  'cahya/xlm-roberta-base-indonesian-NER'],
 'device': device(type='cuda')}

In [17]:
dataset = raw_data

## Train Model


### indobenchmark/indobert-base-p1


In [18]:
MODEL_NAME = config["models"][0]
MODEL_NAME

'indobenchmark/indobert-base-p1'

In [19]:
print(f"\n{'='*50}")
print(f"Training model: {MODEL_NAME}")
print(f"{'='*50}")

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForTokenClassification.from_pretrained(
    MODEL_NAME, 
    num_labels=len(NER_TAG),
    id2label=id2label,
    label2id=label2id
)



Training model: indobenchmark/indobert-base-p1


tokenizer_config.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/229k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

2025-05-16 16:40:03.516068: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747413603.723701      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747413603.785729      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


pytorch_model.bin:   0%|          | 0.00/498M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at indobenchmark/indobert-base-p1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
# Prepare datasets
tokenized_datasets = prepare_datasets(dataset, tokenizer, config)
dataloaders = create_dataloaders(tokenized_datasets, config)



Preparing train dataset...


model.safetensors:   0%|          | 0.00/498M [00:00<?, ?B/s]

Preparing test dataset...
Preparing dev dataset...


In [21]:
# Setup optimizer and scheduler
optimizer = AdamW(model.parameters(), lr=config["learning_rate"])
total_steps = len(dataloaders["train"]) * config["epochs"]
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=config["warmup_steps"],
    num_training_steps=total_steps
)


In [22]:
# Train and evaluate
train_model(model, dataloaders, optimizer, scheduler, config, MODEL_NAME)


Training on cuda

Epoch 1/5


Training: 100%|██████████| 92/92 [00:19<00:00,  4.77it/s, loss=0.512]


Average training loss: 1.1031


Evaluating: 100%|██████████| 23/23 [00:01<00:00, 18.05it/s]


New best F1: 0.1341 - Saving model
Validation Metrics: Precision=0.3241, Recall=0.0845, F1=0.1341, Exact Match= 0.2943

Epoch 2/5


...

New best F1: 0.7948 - Saving model
Validation Metrics: Precision=0.7743, Recall=0.8164, F1=0.7948, Exact Match= 0.7112

Epoch 5/5


Training: 100%|██████████| 92/92 [00:18<00:00,  4.92it/s, loss=0.0409]


Average training loss: 0.0483


Evaluating: 100%|██████████| 23/23 [00:01<00:00, 18.15it/s]


New best F1: 0.8278 - Saving model
Validation Metrics: Precision=0.7888, Recall=0.8708, F1=0.8278, Exact Match= 0.7411


In [23]:
# Load best model
best_model_path = f"models/best_model_{MODEL_NAME.replace('/', '_')}.pt"
if os.path.exists(best_model_path):
    model.load_state_dict(torch.load(best_model_path))


In [24]:
# Evaluate on test set
print("Evaluating on test set...")
test_metrics = evaluate_model(model, dataloaders["test"], config)
print(f"Test Metrics:\nPrecision={test_metrics['precision']:.4f}\nRecall={test_metrics['recall']:.4f}\nF1={test_metrics['f1']:.4f}\nExact Match={test_metrics['exact_match']:.4f}")

Evaluating on test set...


Evaluating: 100%|██████████| 32/32 [00:01<00:00, 18.11it/s]

Test Metrics:
Precision=0.8950
Recall=0.9132
F1=0.9040
Exact Match=0.7878


In [25]:
# Demo predictions
print("\nExample predictions:")
sample_texts = [
    "Presiden Joko Widodo mengunjungi Universitas Indonesia di Depok.",
    "Bank BRI meluncurkan program baru di Jakarta Selatan.",
    "Gubernur DKI Jakarta Anies Baswedan meresmikan MRT Jakarta."
]

for text in sample_texts:
    entities = predict_entities(text, model, tokenizer, config)
    
    print(f"\nText: {text}")
    print("Entities:")
    
    for entity in entities:
        print(f"  - {entity['text']} ({entity['label']})")



Example predictions:

Text: Presiden Joko Widodo mengunjungi Universitas Indonesia di Depok.
Entities:
  - Joko (PER)
  - Joko Widodo (PER)
  - Universitas (LOC)
  - Universitas Indonesia (LOC)
  - Depok. (LOC)

Text: Bank BRI meluncurkan program baru di Jakarta Selatan.
Entities:
  - Bank (ORG)
  - Bank BRI (ORG)
  - Jakarta (LOC)
  - Jakarta Selatan. (LOC)

Text: Gubernur DKI Jakarta Anies Baswedan meresmikan MRT Jakarta.
Entities:
  - Anies (PER)
  - Anies Baswedan (PER)


### xlm-roberta-base


In [26]:
MODEL_NAME = config["models"][1]
MODEL_NAME

'xlm-roberta-base'

In [27]:
print(f"\n{'='*50}")
print(f"Training model: {MODEL_NAME}")
print(f"{'='*50}")

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForTokenClassification.from_pretrained(
    MODEL_NAME, 
    num_labels=len(NER_TAG),
    id2label=id2label,
    label2id=label2id
)



Training model: xlm-roberta-base


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [28]:
# Setup optimizer and scheduler
optimizer = AdamW(model.parameters(), lr=config["learning_rate"])
total_steps = len(dataloaders["train"]) * config["epochs"]
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=config["warmup_steps"],
    num_training_steps=total_steps
)


In [29]:
# Train and evaluate
train_model(model, dataloaders, optimizer, scheduler, config, MODEL_NAME)


Training on cuda

Epoch 1/5


Training: 100%|██████████| 92/92 [00:21<00:00,  4.36it/s, loss=0.608]


Average training loss: 1.4260


Evaluating: 100%|██████████| 23/23 [00:01<00:00, 18.15it/s]
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Validation Metrics: Precision=0.0000, Recall=0.0000, F1=0.0000, Exact Match= 0.3106

Epoch 2/5


...

Validation Metrics: Precision=0.0000, Recall=0.0000, F1=0.0000, Exact Match= 0.3106

Epoch 5/5


Training: 100%|██████████| 92/92 [00:21<00:00,  4.37it/s, loss=0.614]


Average training loss: 0.5867


Evaluating: 100%|██████████| 23/23 [00:01<00:00, 18.09it/s]


New best F1: 0.0071 - Saving model
Validation Metrics: Precision=0.1765, Recall=0.0036, F1=0.0071, Exact Match= 0.3106


In [30]:
# Load best model
best_model_path = f"models/best_model_{MODEL_NAME.replace('/', '_')}.pt"
if os.path.exists(best_model_path):
    model.load_state_dict(torch.load(best_model_path))


In [31]:
# Evaluate on test set
print("Evaluating on test set...")
test_metrics = evaluate_model(model, dataloaders["test"], config)
print(f"Test Metrics:\nPrecision={test_metrics['precision']:.4f}\nRecall={test_metrics['recall']:.4f}\nF1={test_metrics['f1']:.4f}\nExact Match={test_metrics['exact_match']:.4f}")


Evaluating on test set...


Evaluating: 100%|██████████| 32/32 [00:01<00:00, 18.13it/s]

Test Metrics:
Precision=0.1400
Recall=0.0051
F1=0.0098
Exact Match=0.1670


In [32]:
# Demo predictions
print("\nExample predictions:")
sample_texts = [
    "Presiden Joko Widodo mengunjungi Universitas Indonesia di Depok.",
    "Bank BRI meluncurkan program baru di Jakarta Selatan.",
    "Gubernur DKI Jakarta Anies Baswedan meresmikan MRT Jakarta."
]

for text in sample_texts:
    entities = predict_entities(text, model, tokenizer, config)
    
    print(f"\nText: {text}")
    print("Entities:")
    
    for entity in entities:
        print(f"  - {entity['text']} ({entity['label']})")



Example predictions:

Text: Presiden Joko Widodo mengunjungi Universitas Indonesia di Depok.
Entities:
  - Presiden (ORG)
  - Widodo (ORG)

Text: Bank BRI meluncurkan program baru di Jakarta Selatan.
Entities:
  - di (ORG)

Text: Gubernur DKI Jakarta Anies Baswedan meresmikan MRT Jakarta.
Entities:


### cahya/xlm-roberta-base-indonesian-NER


In [33]:
MODEL_NAME = config["models"][2]
MODEL_NAME

'cahya/xlm-roberta-base-indonesian-NER'

In [34]:
print(f"\n{'='*50}")
print(f"Training model: {MODEL_NAME}")
print(f"{'='*50}")

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

custom_config = AutoConfig.from_pretrained(
    MODEL_NAME,
    num_labels=len(NER_TAG),
    id2label=id2label,
    label2id=label2id
)

# Load model
model = AutoModelForTokenClassification.from_pretrained(
    MODEL_NAME,
    config=custom_config,
    ignore_mismatched_sizes=True
)



Training model: cahya/xlm-roberta-base-indonesian-NER


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/2.05k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

Some weights of the model checkpoint at cahya/xlm-roberta-base-indonesian-NER were not used when initializing XLMRobertaForTokenClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at cahya/xlm-roberta-base-indonesian-NER and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([39]) in the checkpoint and torch.Size([7]) in the model 

In [35]:
# Setup optimizer and scheduler
optimizer = AdamW(model.parameters(), lr=config["learning_rate"])
total_steps = len(dataloaders["train"]) * config["epochs"]
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=config["warmup_steps"],
    num_training_steps=total_steps
)


In [36]:
# Train and evaluate
train_model(model, dataloaders, optimizer, scheduler, config, MODEL_NAME)


Training on cuda

Epoch 1/5


Training: 100%|██████████| 92/92 [00:21<00:00,  4.37it/s, loss=0.718]


Average training loss: 1.2160


Evaluating: 100%|██████████| 23/23 [00:01<00:00, 18.21it/s]


Validation Metrics: Precision=0.0000, Recall=0.0000, F1=0.0000, Exact Match= 0.3106

Epoch 2/5


...

Validation Metrics: Precision=0.0000, Recall=0.0000, F1=0.0000, Exact Match= 0.3106

Epoch 5/5


Training: 100%|██████████| 92/92 [00:21<00:00,  4.37it/s, loss=0.507]


Average training loss: 0.5995


Evaluating: 100%|██████████| 23/23 [00:01<00:00, 18.20it/s]


Validation Metrics: Precision=0.0000, Recall=0.0000, F1=0.0000, Exact Match= 0.3106


In [37]:
# Load best model
best_model_path = f"models/best_model_{MODEL_NAME}.pt"
if os.path.exists(best_model_path):
    model.load_state_dict(torch.load(best_model_path))


In [38]:
# Evaluate on test set
print("Evaluating on test set...")
test_metrics = evaluate_model(model, dataloaders["test"], config)
print(f"Test Metrics:\nPrecision={test_metrics['precision']:.4f}\nRecall={test_metrics['recall']:.4f}\nF1={test_metrics['f1']:.4f}\nExact Match={test_metrics['exact_match']:.4f}")

Evaluating on test set...


Evaluating: 100%|██████████| 32/32 [00:01<00:00, 18.28it/s]

Test Metrics:
Precision=0.0000
Recall=0.0000
F1=0.0000
Exact Match=0.1670


In [39]:
# Demo predictions
print("\nExample predictions:")
sample_texts = [
    "Presiden Joko Widodo mengunjungi Universitas Indonesia di Depok.",
    "Bank BRI meluncurkan program baru di Jakarta Selatan.",
    "Gubernur DKI Jakarta Anies Baswedan meresmikan MRT Jakarta."
]

for text in sample_texts:
    entities = predict_entities(text, model, tokenizer, config)
    
    print(f"\nText: {text}")
    print("Entities:")
    
    for entity in entities:
        print(f"  - {entity['text']} ({entity['label']})")



Example predictions:

Text: Presiden Joko Widodo mengunjungi Universitas Indonesia di Depok.
Entities:
  - Joko (LOC)
  - Universitas (I-ORG)

Text: Bank BRI meluncurkan program baru di Jakarta Selatan.
Entities:

Text: Gubernur DKI Jakarta Anies Baswedan meresmikan MRT Jakarta.
Entities:
  - Anies (LOC)


## Hyper Parameter Tuning


In [40]:
best_scores = {}

def hyperparameter_tuning(base_config, search_space):
    keys, values = zip(*search_space.items())
    experiments = [dict(zip(keys, v)) for v in product(*values)]

    for i, params in enumerate(experiments):
        print(f"\n======== Running Experiment {i+1}/{len(experiments)} ========")
        config = base_config.copy()
        config.update(params)

        for MODEL_NAME in config["models"]:
        # for MODEL_NAME in ["cahya/xlm-roberta-base-indonesian-NER"]:
            print(f"\n{'='*50}")
            print(f"Training model: {MODEL_NAME} with params: {params}")
            print(f"{'='*50}")

            tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

            if MODEL_NAME == 'cahya/xlm-roberta-base-indonesian-NER':
                # Load tokenizer and model
                custom_config = AutoConfig.from_pretrained(
                    MODEL_NAME,
                    num_labels=len(NER_TAG),
                    id2label=id2label,
                    label2id=label2id
                )
                
                # Load model
                model = AutoModelForTokenClassification.from_pretrained(
                    MODEL_NAME,
                    config=custom_config,
                    ignore_mismatched_sizes=True
                )


            else:
                model = AutoModelForTokenClassification.from_pretrained(
                    MODEL_NAME, 
                    num_labels=len(NER_TAG),
                    id2label=id2label,
                    label2id=label2id
                )
                
            model.to(config["device"])

            tokenized_datasets = prepare_datasets(dataset, tokenizer, config)
            dataloaders = create_dataloaders(tokenized_datasets, config)

            optimizer = AdamW(model.parameters(), lr=config["learning_rate"])
            total_steps = len(dataloaders["train"]) * config["epochs"]
            scheduler = get_linear_schedule_with_warmup(
                optimizer,
                num_warmup_steps=config["warmup_steps"],
                num_training_steps=total_steps
            )

            train_model(model, dataloaders, optimizer, scheduler, config, MODEL_NAME)

            model_type = model.config.model_type
            exp_name = f"{model_type}_lr{config['learning_rate']}_bs{config['batch_size']}_ep{config['epochs']}"
            model_path = f"models/{exp_name}.pt"
            os.makedirs("models", exist_ok=True)
            torch.save(model.state_dict(), model_path)

            model.load_state_dict(torch.load(model_path))
            print(f"\nEvaluating {exp_name} on test set...")
            test_metrics = evaluate_model(model, dataloaders["test"], config)
            print(f"Metrics: Precision={test_metrics['precision']:.4f}, Recall={test_metrics['recall']:.4f}, F1={test_metrics['f1']:.4f}, Exact Match={test_metrics['exact_match']:.4f}")

            current_best = best_scores.get(model_type, {"f1": 0})
            
            if test_metrics["f1"] > current_best["f1"]:
                best_model_path = f"models/best_{model_type}.pt"
                torch.save(model.state_dict(), best_model_path)

                save_dir = f"saved_models/{MODEL_NAME.replace('/', '_')}"
                os.makedirs(save_dir, exist_ok=True)
                model.save_pretrained(save_dir)
                tokenizer.save_pretrained(save_dir)
            
                with open(os.path.join(save_dir, "label2id.json"), "w") as f:
                    json.dump(label2id, f)
                with open(os.path.join(save_dir, "id2label.json"), "w") as f:
                    json.dump(id2label, f)

                best_scores[model_type] = {"f1": test_metrics["f1"], "path": best_model_path, "params": params}
                print(f"New best model saved for {model_type} at {best_model_path}")


In [41]:
base_config = get_config()
search_space = {
    "learning_rate": [10e-5, 5e-5, 3e-5, 2e-5],
    "batch_size": [16, 32],
    "epochs": [3, 5, 10, 20],
}

hyperparameter_tuning(base_config, search_space)



======== Running Experiment 1/32 ========

Training model: indobenchmark/indobert-base-p1 with params: {'learning_rate': 0.0001, 'batch_size': 16, 'epochs': 3}


Some weights of BertForTokenClassification were not initialized from the model checkpoint at indobenchmark/indobert-base-p1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Preparing train dataset...
Preparing test dataset...
Preparing dev dataset...
Training on cuda

Epoch 1/3


Training: 100%|██████████| 92/92 [00:18<00:00,  4.92it/s, loss=0.296] 


Average training loss: 0.7052


...

Training: 100%|██████████| 92/92 [00:21<00:00,  4.38it/s, loss=0.0729]


Average training loss: 0.1200


Evaluating: 100%|██████████| 23/23 [00:01<00:00, 18.26it/s]


New best F1: 0.8255 - Saving model
Validation Metrics: Precision=0.7937, Recall=0.8600, F1=0.8255, Exact Match= 0.7357


RuntimeError: [enforce fail at inline_container.cc:626] . unexpected pos 43968 vs 43862

In [ ]:
# # Inference

# from transformers import AutoTokenizer, AutoModelForTokenClassification
# import json

# load_dir = "saved_model"

# tokenizer = AutoTokenizer.from_pretrained(load_dir)
# model = AutoModelForTokenClassification.from_pretrained(load_dir)

# with open(f"{load_dir}/label2id.json") as f:
#     label2id = json.load(f)
# with open(f"{load_dir}/id2label.json") as f:
#     id2label = json.load(f)
!ls models

In [ ]:
# dataset used
"""
https://github.com/khairunnisaor/idner-news-2k/tree/main

@inproceedings{khairunnisa2020,
  title={Towards a Standardized Dataset on Indonesian Named Entity Recognition},
  author={Siti Oryza Khairunnisa and Aizhan Imankulova and Mamoru Komachi},
  booktitle={Proceedings of the 1st Conference of the Asia-Pacific Chapter of the Association for Computational Linguistics and the 10th International Joint Conference on Natural Language Processing: Student Research Workshop}
  year={2020}
}
"""